In [1]:
import random  
import operator
from pyspark import SparkContext, SparkConf
import numpy as np


#Configure pyspark
conf = SparkConf().setMaster("local[*]")
sc = SparkContext(conf=conf)



In [2]:
class Triest_improved():
    def __init__(self, M):
        self.S = set()
        self.t = 0
        self.tau = 0
        self.tau_count = {}
        self.M = M
        
    def flippedBiasedCoin(self):
        hot=random.choices(population = ["heads","tail"], weights=[self.M/self.t,1-(self.M/self.t)])
        return hot[0]          
            
            
    def sampleEdge(self,edge):
        if self.t<=self.M:
            return True
        elif self.flippedBiasedCoin()=="heads":
            random_edge = random.sample(self.S,1)[0]
            self.S.remove(random_edge)
            #self.updateCounters(operator.sub, random_edge)
            return True
        else:
            return False

    def updateCounters(self,operator,edge):
        u=edge[0]
        v=edge[1]
        #u = edge.split("-")[0]
        #v = edge.split("-")[1]
        nu = set()
        nv = set()
        #Alla neighbours
        #rdd = sc.parallelize(self.S)
        #nu = set(rdd.filter(lambda x: x[0] == u or x[1] == u ).map(lambda x: x[0] if x[1] == u else x[1]).collect())
        #nv = set(rdd.filter(lambda x: x[0] == v or x[1] == v ).map(lambda x: x[0] if x[1] == v else x[1]).collect())
        nu = set([elem[0] if elem[0] == u else elem[1] for elem in self.S if (elem[0] == u or elem[1] == u)])
        nv = set([elem[0] if elem[0] == v else elem[1] for elem in self.S if (elem[0] == v or elem[1] == v)])
        #nv = [elem[0] for elem in self.S if elem[0] == v elif elem[1] if elem[1] == v]
        #for elem in self.S:
        #    if u == elem[0]:
        #        nu.add(elem[1])
        #    if u == elem[1]:
        #        nu.add(elem[0])
        #    if v == elem[0]:
        #        nv.add(elem[1])
        #    if v == elem[1]:
        #        nv.add(elem[0])
        #Loopa igenom alla snitten av neighbours till u och v        
        #print(nu.intersection(nv))
        n = (self.t-1)*(self.t-2)
        n_t = n/(self.M*(self.M-1))
        if n_t < 1:
            n_t = 1
        for c in nu.intersection(nv):
            self.tau = operator(self.tau,n_t)
            self.tau_count[c] = operator(int(self.tau_count.get(c) or 0),n_t)
            self.tau_count[u] = operator(int(self.tau_count.get(u) or 0),n_t)
            self.tau_count[v] = operator(int(self.tau_count.get(v) or 0),n_t)
    def start(self, dataset):
        for edge in dataset:
            self.t = self.t + 1
            if self.t % 2000 == 0:
                print("t:",self.t,"tau:",self.tau)
            self.updateCounters(operator.add,edge)
            if self.sampleEdge(edge):
                self.S.add(edge)  
        #return self.tau


In [3]:

rdd = sc.textFile("web-NotreDame.txt")
dataset = rdd.filter(lambda line: "#" not in line) \
             .map(lambda line: tuple([int(x) for x in line.split('\t')])) \
             .map(lambda edge: edge[::-1] if edge[0] > edge[1] else edge) \
             .filter(lambda edge: edge[0] != edge[1]).distinct()

In [4]:
impr = Triest_improved(dataset.count())
print("The number of triangles are:", dataset.map(impr.start))


The number of triangles are: PythonRDD[7] at RDD at PythonRDD.scala:53
